#하와이 데이터 탐색


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import requests
import json
import pandas as pd

#하와이 리뷰 데이터

### df에 담기

In [0]:
import json
import gzip

path = '/content/drive/MyDrive/T아카데미_ASAC프로젝트_5기_멘토링/3. 데이터/review-Hawaii_large.json'

def parse1(path):
    if path.endswith('.gz'):
        g = gzip.open(path, 'rt')  # gzip 파일일 경우
    else:
        g = open(path, 'r')  # 일반 텍스트 파일일 경우
    for l in g:
        yield json.loads(l)
    g.close()

json_objects = list(parse1(path))

# 첫 번째 JSON 객체를 출력하여 확인
print(json.dumps(json_objects[0], indent=4))
print(f"성공적으로 로드된 JSON 객체 수: {len(json_objects)}")


In [0]:
type(json_objects)

In [0]:
json_objects[0]

In [0]:
# 데이터를 리스트에 저장
data = []
for item in json_objects:
    data.append(item)

# 데이터를 판다스 데이터프레임으로 변환
df = pd.DataFrame(data)

# 데이터프레임의 첫 몇 줄을 출력하여 확인
print(df.head())


In [0]:
df.head()

### df.info()

In [0]:
df.info()

### 결측치

In [0]:
df.loc[df['text'].isnull()]

In [0]:
print('총갯수',len(df))
for i in df.columns:

  print(i,':',df[i].isnull().sum())

### 사진

In [0]:
df.head()

In [0]:
# df[len(df['pics']) > 3]
# df[df['pics'].apply(len) > 3]

In [0]:
df['pics_cnt'] = 0

In [0]:
for i,v in enumerate(df['pics']):
  if v == None:
    df.at[i,'pics_cnt'] = 0
  else :
    df.at[i,'pics_cnt'] = len(df['pics'].iloc[i])

In [0]:
df.head()

In [0]:
df.to_parquet('hawaii_review.parquet', index=False)

In [0]:
!cp hawaii_review.parquet /content/drive/MyDrive/

In [0]:
non_zero_count = (df['pics_cnt'] != 0).sum()
print(non_zero_count)

In [0]:
print(len(df['pics_cnt']))
print(len(df['pics']))

In [0]:
df['pics_cnt'].sum()/non_zero_count

In [0]:
df[df['pics_cnt']==df['pics_cnt'].max()].head()

In [0]:
# df[df['pics_cnt']==5]
pics_test = df.at[36,'pics']

In [0]:
pics_test[0]['url']

In [0]:
len(df.at[581,'pics'])

- 리뷰에 포함된 사진의 개수는 최소 0, 최대 50
- 사진이 포함된 리뷰를 기준으로 사진의 개수는 약 3장

In [0]:
import matplotlib.pyplot as plt

In [0]:
plt.figure(figsize=(10, 6))
plt.hist(df['pics_cnt'], bins=10, edgecolor='black')
plt.xlabel('pics_cnt')
plt.ylabel('Frequency')
plt.show()

### 시간

In [0]:
# 시간 컬럼을 사람이 읽을 수 있는 형식으로 변환
df['time'] = pd.to_datetime(df['time'], unit='ms')

# 결측값 처리 (여기서는 간단히 None을 빈 문자열로 대체)
df['pics'] = df['pics'].apply(lambda x: x if x is not None else [])
df['resp'] = df['resp'].fillna('')

# 데이터 정리: 텍스트 컬럼의 줄바꿈 문자 제거
df['text'] = df['text'].str.replace('\n', ' ')

# 필요하면 데이터 타입 변환 (예: rating 컬럼을 float로)
df['rating'] = df['rating'].astype(float)

# 데이터 확인
print(df.head())

In [0]:
df.head()

In [0]:
from tqdm import tqdm

In [0]:
for i in tqdm(range(len(df))):
  df.at[i,'year'] = str(df['time'][i])[:4]

In [0]:
year_counts = df['year'].value_counts().sort_index()
year_counts.plot(kind='bar', figsize=(10, 6))
plt.xlabel('Year')
plt.ylabel('Frequency')
plt.title('Yearly Distribution')
plt.show()

In [0]:
df[df['year']=='1990']

In [0]:
# 날짜 문자열을 요일로 변환하는 함수
def get_weekday_from_string(date_string):
    date = pd.to_datetime(date_string)
    weekdays = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    return weekdays[date.weekday()]

# 새로운 열 추가
df['Weekday'] = df['time'].apply(get_weekday_from_string)

In [0]:
df['Weekday'].value_counts()

In [0]:
# 요일별 리뷰 수 계산
weekday_counts = df['Weekday'].value_counts().reindex(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])

# 막대 그래프로 시각화
plt.figure(figsize=(10, 6))
weekday_counts.plot(kind='bar', color='skyblue')
plt.title('Number of Reviews by Weekday')
plt.xlabel('Weekday')
plt.ylabel('Number of Reviews')
plt.xticks(rotation=0)
plt.grid(axis='y')

# 그래프 출력
plt.show()

In [0]:
print(df.groupby('Weekday')['rating'].mean())

In [0]:
# 요일별 평점 평균 계산
weekday_avg_rating = df.groupby('Weekday')['rating'].mean().reindex(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])

# 막대 그래프로 시각화
plt.figure(figsize=(10, 6))
weekday_avg_rating.plot(kind='bar', color='skyblue')
plt.title('Average Rating by Weekday')
plt.xlabel('Weekday')
plt.ylabel('Average Rating')
plt.ylim(0, 5)  # 평점 범위 (0-5)
plt.xticks(rotation=0)
plt.grid(axis='y')

# 그래프 출력
plt.show()

In [0]:
# 요일별 평점 평균 계산
weekday_avg_rating = df.groupby('Weekday')['rating'].mean().reindex(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])

# 막대 그래프로 시각화
plt.figure(figsize=(10, 6))
bars = weekday_avg_rating.plot(kind='bar', color='skyblue')
plt.title('Average Rating by Weekday')
plt.xlabel('Weekday')
plt.ylabel('Average Rating')
plt.ylim(0, 5)  # 평점 범위 (0-5)
plt.xticks(rotation=0)
plt.grid(axis='y')

# 막대 상단에 값 표시
for bar in bars.patches:
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() - 0.1,
        f'{bar.get_height():.2f}',
        ha='center',
        va='bottom',
        fontsize=10
    )

# 그래프 출력
plt.show()

### 텍스트

In [0]:
df[df['text'].notnull()].head()

In [0]:
type(df.at[0,'text'])

In [0]:
# 바다와 관련된 영단어 리스트
sea_words = ['ocean', 'beach', 'sea', 'waves', 'sand', 'coast', 'shore']

# None 값을 빈 문자열로 대체
df['text'] = df['text'].fillna('')

# beach 열 추가 및 값 설정
df['beach'] = df['text'].apply(lambda x: 1 if any(word in x.split() for word in sea_words) else 0)

In [0]:
df.at[7,'text']

In [0]:
df['beach'].value_counts()

In [0]:
import matplotlib.pyplot as plt

# beach 열을 기준으로 평점 평균 계산
beach_mean = df[df['beach'] == 1]['rating'].mean()
non_beach_mean = df[df['beach'] == 0]['rating'].mean()

overall_mean = df['rating'].mean()

# 시각화
labels = ['Beach Related', 'Not Beach Related']
means = [beach_mean, non_beach_mean]

plt.figure(figsize=(8, 6))
bars = plt.bar(labels, means, color=['blue', 'orange'])
plt.xlabel('Category')
plt.ylabel('Average Rating')
plt.title('Average Rating by Beach Related Content')
plt.ylim(0, 5)  # y축 범위를 0에서 5로 설정

# 전체 평균을 표시하는 수평선 추가
plt.axhline(y=overall_mean, color='red', linestyle='--', label=f'Overall Mean: {overall_mean:.3f}')

# 막대 위에 값을 표시
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2.0, yval, round(yval, 3), ha='center', va='bottom')

# 범례 추가
plt.legend()

plt.show()


In [0]:
df['rating'].mean()

In [0]:
# 사람과 관련된 단어 리스트
people_words = ['family', 'child', 'children', 'kids', 'couple', 'friend', 'friends', 'baby', 'parents']

# None 값을 빈 문자열로 대체
df['text'] = df['text'].fillna('')

# people 열 추가 및 값 설정
df['people'] = df['text'].apply(lambda x: 1 if any(word in x for word in people_words) else 0)


In [0]:
# people 열을 기준으로 평점 평균 계산
people_mean = df[df['people'] == 1]['rating'].mean()
non_people_mean = df[df['people'] == 0]['rating'].mean()

# 시각화
labels = ['People Related', 'Not People Related']
means = [people_mean, non_people_mean]

plt.figure(figsize=(8, 6))
bars = plt.bar(labels, means, color=['green', 'red'])
plt.xlabel('Category')
plt.ylabel('Average Rating')
plt.title('Average Rating by People Related Content')
plt.ylim(0, 5)  # y축 범위를 0에서 5로 설정

# 각 막대 위에 값 표시
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 3), ha='center', va='bottom')

plt.show()

In [0]:
df['people'].value_counts()

#하와이 메타 데이터


In [0]:
!mkdir unzip_path
!cd unzip_path # 압축을 풀 폴더 경로
!unzip -qq '/content/drive/MyDrive/ASAC/Project_GoogleMap/meta-Hawaii.json.gz' # 압축파일 경로

In [0]:
import gzip
import json
import pandas as pd

# JSON 데이터를 파싱하는 함수
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield json.loads(l)

# 데이터 경로
data_path = '/content/drive/MyDrive/ASAC/Project_GoogleMap/meta-Hawaii.json.gz'

# 데이터를 리스트에 저장
data = []
for item in parse(data_path):
    data.append(item)

# 데이터를 판다스 데이터프레임으로 변환
df_meta = pd.DataFrame(data)

# 데이터프레임의 첫 몇 줄을 출력하여 확인
print(df_meta.head())

# 데이터프레임을 저장할 수도 있습니다.
#df.to_csv('/content/drive/MyDrive/korean_credit_data/meta-Hawaii.csv', index=False)


In [0]:
data

In [0]:
for i in df_meta['price'].loc[df_meta['price'].isnull()==False]:
  print(i)

In [0]:
df_meta.shape

In [0]:
df_meta.info()

In [0]:
print('총갯수',len(df_meta))
for i in df_meta.columns:

  print(i,':',df_meta[i].isnull().sum())

In [0]:
for i in df_meta['state'][:100]:
  print(i)

In [0]:
df_meta.head()

### 카테고리

In [0]:
df_meta['category'].value_counts()

In [0]:
df_meta['category'].value_counts().mean()

In [0]:
import pandas as pd
import matplotlib.pyplot as plt

# df_meta 데이터프레임이 이미 존재한다고 가정합니다.

# category 컬럼이 리스트인 경우 문자열로 변환
df_meta['category'] = df_meta['category'].apply(lambda x: ''.join(x) if isinstance(x, list) else x)

# category 컬럼 값들의 빈도 계산
category_counts = df_meta['category'].value_counts()

# category 값의 수의 평균 계산
# average_frequency = category_counts.mean()

# 빈도가 평균 이상인 category 값들 필터링
frequent_categories = category_counts[category_counts >= 100]

# 그래프 생성
plt.figure(figsize=(10, 6))
plt.bar(frequent_categories.index, frequent_categories.values, color='skyblue')
plt.xlabel('Category')
plt.ylabel('Frequency')
plt.title('Categories with Frequency Above 100', fontsize=16)
plt.xticks(rotation=45, ha='right', fontsize=12)  # x라벨 회전 및 글꼴 크기 조정
plt.tight_layout()  # 레이아웃 조정
plt.show()

In [0]:
# category 컬럼이 리스트인 경우 문자열로 변환
df_meta['category'] = df_meta['category'].apply(lambda x: ''.join(x) if isinstance(x, list) else x)

# category 컬럼 값들의 빈도 계산
category_counts = df_meta['category'].value_counts()

# 빈도가 100 이상인 category 값들 필터링
frequent_categories = category_counts[category_counts >= 100].index

# 빈도가 100 이상인 카테고리들의 평점 평균 계산
category_ratings = df_meta[df_meta['category'].isin(frequent_categories)].groupby('category')['avg_rating'].mean()

# 평점 평균이 높은 순서대로 정렬
category_ratings = category_ratings.sort_values(ascending=False)

# 그래프 생성
plt.figure(figsize=(12, 8))  # 그래프 크기 조정
bars = plt.bar(category_ratings.index, category_ratings.values, color='skyblue')
plt.xlabel('Category', fontsize=14)
plt.ylabel('Average Rating', fontsize=14)
plt.title('Average Rating by Category (Frequency >= 100)', fontsize=16)
plt.xticks(rotation=45, ha='right', fontsize=12)  # x라벨 회전 및 글꼴 크기 조정
plt.ylim(3, max(category_ratings.values) + 0.5)  # y축 범위 설정

# 막대 위에 평점 값 표시
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.05, round(yval, 2), ha='center', va='bottom', fontsize=12)

plt.tight_layout()  # 레이아웃 조정
plt.show()

In [0]:
# category 컬럼이 리스트인 경우 문자열로 변환
df_meta['category'] = df_meta['category'].apply(lambda x: ''.join(x) if isinstance(x, list) else x)

# category 컬럼 값들의 빈도 계산
category_counts = df_meta['category'].value_counts()

# 빈도가 100 이상인 category 값들 필터링
frequent_categories = category_counts[category_counts >= 100].index

# 빈도가 100 이상인 카테고리들의 평점 통계 계산
category_stats = df_meta[df_meta['category'].isin(frequent_categories)].groupby('category')['avg_rating'].agg(
    min_rating='min',
    max_rating='max',
    mean_rating='mean',
    mode_rating=lambda x: x.mode()[0] if not x.mode().empty else None
).reset_index()

# 리스트로 변환
stats_list = category_stats.values.tolist()

# 리스트를 데이터프레임으로 변환
category_stats_df = pd.DataFrame(stats_list, columns=['category', 'min_rating', 'max_rating', 'mean_rating', 'mode_rating'])
category_stats_df_sorted = category_stats_df.sort_values(by='mean_rating', ascending=False).reset_index(drop=True)

category_stats_df_sorted

### 리뷰 수

In [0]:
# num_of_reviews의 분포 시각화
plt.figure(figsize=(10, 6))
plt.hist(df_meta['num_of_reviews'].dropna(), bins=30, edgecolor='black')
plt.title('Distribution of Number of Reviews')
plt.xlabel('Number of Reviews')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [0]:
plt.figure(figsize=(10, 6))
plt.hist(df_meta['num_of_reviews'].dropna(), bins=30, edgecolor='black')
plt.title('Distribution of Number of Reviews')
plt.xlabel('Number of Reviews')
plt.ylabel('Frequency')
plt.grid(True)
plt.xlim(0, 2000)  # x축 범위를 0에서 2000으로 설정
plt.show()

In [0]:
# num_of_reviews의 박스 플롯 시각화
plt.figure(figsize=(10, 6))
plt.boxplot(df_meta['num_of_reviews'].dropna(), vert=False)
plt.title('Boxplot of Number of Reviews')
plt.xlabel('Number of Reviews')
plt.grid(True)
plt.show()

### MISC